In [20]:
import pandas as pd
import numpy as np
import os

In [21]:
# this will be used to access the bigquery client
from google.cloud import bigquery
# this will be used to access the service account modules - passwords and access tokens
from google.oauth2 import service_account
# this will be used for returning data, increases the speed
from google.cloud import bigquery_storage
# for os library
import os


#general dataframe usage
import pandas as pd
# required for certain returns
import pandas_gbq
# exception for a short script
from google.api_core.exceptions import NotFound

In [22]:
# Path to service account if stored locally
SERVICE_ACCOUNT_JSON = r"..\portfolio2026-485323-70c4d609b156.json"

# Credentials to access Big Query via the service account
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_JSON)

# BigQuery client
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

# BigQuery Storage client
bq_storage_client = bigquery_storage.BigQueryReadClient(credentials=credentials)

In [23]:
datasets = list(client.list_datasets())

In [24]:
if datasets:
    print("Datasets in project:")
    for dataset in datasets:
        print(dataset.dataset_id)
else:
    print("No datasets found.")


# function to see if table already exists in big query data set
def table_exists(client, full_table_id):
    try:
        client.get_table(full_table_id)
        return True
    except NotFound:
        return False

Datasets in project:
portfolio1


# Read In Files

In [25]:
pd.read_csv(r'data_files\nba2024.txt')

,Rk,Player,Age,Team,Pos,G,GS,MP,FG,FGA,...,TRB,AST,STL,BLK,TOV,PF,PTS,Trp-Dbl,Awards,Player-additional
0,1,Shai Gilgeous-Alexander,26,OKC,PG,76,76,2598,860,1656,...,379,486,131,77,183,164,2484,0,NaN,gilgesh01
1,2,Anthony Edwards,23,MIN,SG,79,79,2871,721,1612,...,450,359,91,51,249,150,2177,0,NaN,edwaran01
2,3,Nikola Jokić,29,DEN,C,70,70,2571,786,1364,...,892,716,127,45,230,160,2071,34,NaN,jokicni01
3,4,Giannis Antetokounmpo,30,MIL,PF,67,67,2289,793,1319,...,798,433,58,78,206,155,2036,11,NaN,antetgi01
4,5,Jayson Tatum,26,BOS,PF,72,72,2624,662,1465,...,623,431,76,38,209,157,1932,2,NaN,tatumja01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730,565,Riley Minix,24,SAS,SF,1,0,7,0,1,...,2,0,0,0,0,0,0,0,NaN,minixri01
731,566,Jahlil Okafor,29,IND,C,1,0,3,0,0,...,1,1,0,0,0,0,0,0,NaN,okafoja01
732,567,Zyon Pullin,23,MEM,SG,3,0,3,0,1,...,0,0,0,0,0,0,0,0,NaN,pullizy01
733,568,Isaiah Stevens,24,MIA,PG,3,0,6,0,2,...,2,0,1,0,0,0,0,0,NaN,steveis01


In [26]:
dataframes = []

row_count = 0

for dirpath, dirs, files in os.walk(r"data_files"):
    for n, file in enumerate(files):
        df = pd.read_csv(os.path.join(dirpath,file))
        df = df.drop('Player-additional', axis=1)
        df['Year'] = 2020 + n + 1
        if 'MP▼' in df.columns:
            df.rename(columns={'MP▼':'MP'}, inplace=True)

        if 'Team' in df.columns:
            df.rename(columns={'Team':'Tm'}, inplace=True) 
        display(df.shape)
        dataframes.append(df)
        row_count += df.shape[0]

nba_df = pd.concat(dataframes)

(705, 31)

(812, 31)

(679, 31)

(735, 33)

## Test for shape accuracy

In [27]:
assert nba_df.shape[0] == row_count, f"Expected {row_count} rows, but got {nba_df.shape[0]}"

# Data Manipulation - Cleaning

### Player Codes

In [28]:
player_codes = {}

for n,player in enumerate(set(nba_df['Player'])):
    if player not in player_codes.keys():
        player_codes[player] = int(f"100{n}")

nba_df['Player Codes'] = nba_df['Player'].map(player_codes)

nba_df.drop(columns = 'Rk', inplace=True)

In [29]:
assert nba_df['Player'].nunique() == nba_df['Player Codes'].nunique(), f"Expected {nba_df['Player'].nunique()}, player codes, but got {nba_df['Player Codes'].nunique()}"

### Totals

In [30]:
nba_totals = nba_df[['Player', 'Tm','Pos', 'Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year']].copy()

In [31]:
#drop the total rows to avoid double counting
nba_totals = nba_totals[~(nba_totals['Tm'] == 'TOT') & ~(nba_totals['Tm'] == '2TM') & ~(nba_totals['Tm'] == '3TM')].copy()

In [32]:
# averages need to be dropped as they will need to be recalculated based on the total attempts and makes
nba_totals.drop(columns=['3P%','2P%','eFG%','FT%'],inplace=True)

In [33]:
def smart_agg(x):
    if np.issubdtype(x.dtype, np.number):
        if x.dtype == float:
            return x.mean()
        else:
            return x.sum()
    else:
        return x.mode()[0]  # or .mode()[0], .unique(), etc. - helpful for the Team object/nominal data

aggregated_totals = nba_totals.groupby(['Player', 'Age', 'Year']).agg(lambda x: smart_agg(x)).reset_index()

In [34]:
aggregated_totals.head()

,Player,Age,Year,Tm,Pos,G,GS,MP,FG,FGA,...,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,A.J. Green,23,2023,MIL,SG,35,1,345,53,125,...,4,6,39,45,22,6,0,9,31,154
1,A.J. Green,25,2024,MIL,SG,73,7,1659,182,424,...,27,18,156,174,108,37,7,40,157,541
2,A.J. Lawson,22,2023,DAL,SG,15,0,108,22,44,...,8,6,15,21,2,2,0,3,11,56
3,A.J. Lawson,24,2024,TOR,SG,26,2,486,80,190,...,63,20,66,86,31,13,6,15,44,236
4,AJ Griffin,19,2023,ATL,SF,72,12,1401,248,533,...,47,37,116,153,73,42,12,42,87,639


# Write to Big Query

In [37]:
aggregated_totals.columns = aggregated_totals.columns.str.lower().str.strip().str.replace(r"[^\w]+","_", regex=True)

In [38]:
# Write to bigquery

# Write truncate will overwrite the table instead of appending
job_config = bigquery.LoadJobConfig(write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
                                    autodetect=True)

full_table_id = f"{credentials.project_id}.portfolio1.aggregated_totals"

if table_exists(client, full_table_id):
    print(f"Table {full_table_id} already exists.")
    print("Overwriting existing table...")
else:
    print("Creating new table...")

load_job = client.load_table_from_dataframe(aggregated_totals, full_table_id, job_config=job_config)
load_job.result()

Table portfolio2026-485323.portfolio1.aggregated_totals already exists.
Overwriting existing table...


LoadJob<project=portfolio2026-485323, location=US, id=e6379741-7652-4d35-a5a8-3c54795a0985>